# Exemple

In [2]:
import mlflow

In [3]:
#!pip freeze > /mnt/docker/requirements.txt

## Utilisation du package

In [4]:
#Cette cellule permet d'appeler la version packagée du projet et d'en assurer le reload avant appel des fonctions
%load_ext autoreload
%autoreload 2

# Configuration de l'experiment MLFlow

In [5]:
mlflow.tracking.get_tracking_uri()

'/mnt/experiments'